In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.ops import nearest_points # Para distancia a líneas (carreteras)
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
# --- Rutas y Nombres de Archivo (Configurables) ---
RUTA_TIENDAS_CON_CENSO_DENUE_INPUT = './datos/output/tiendas_train_censo_denue.gpkg'
LAYER_TIENDAS_INPUT = 'tiendas_censo_denue' # Capa dentro del gpkg anterior

RUTA_OSM_DATA = './datos/osm_data/' # Donde están tus shapefiles de OSM
SHP_OSM_POIS = RUTA_OSM_DATA + 'gis_osm_pois_a_free_1.shp' 
SHP_OSM_ROADS = RUTA_OSM_DATA + 'gis_osm_roads_free_1.shp'

# Columnas clave en los shapefiles de OSM (usualmente 'fclass' y 'name')
COL_OSM_FCLASS = 'fclass'
COL_OSM_NAME = 'name'

# CRS de las tiendas (EPSG:6372) y CRS original de OSM (usualmente WGS84, EPSG:4326)
CRS_OBJETIVO = 'EPSG:6372'
CRS_OSM_ORIGINAL = 'EPSG:4326'

# Radios para análisis de conteo de PDIs OSM (en metros)
RADIOS_ANALISIS_M = [200, 500, 1000] # Puedes ajustar o añadir más

# Archivo de salida
RUTA_DATOS_OUTPUT = './datos/output/'
ARCHIVO_SALIDA_GPKG = RUTA_DATOS_OUTPUT + 'tiendas_train_censo_denue_osm.gpkg'
# --- Fin de Configuración ---

print("--- Iniciando Integración de Datos de OpenStreetMap (OSM) ---")

In [ ]:
# --- 1. Cargar el GeoDataFrame de Tiendas (ya con Censo y DENUE) ---
try:
    tiendas_gdf = gpd.read_file(RUTA_TIENDAS_CON_CENSO_DENUE_INPUT, layer=LAYER_TIENDAS_INPUT)
    print(f"GeoDataFrame de tiendas con censo y DENUE cargado: {len(tiendas_gdf)} tiendas.")
    if tiendas_gdf.crs.to_string().upper() != CRS_OBJETIVO.upper():
        print(f"Reproyectando tiendas_gdf a {CRS_OBJETIVO}...")
        tiendas_gdf = tiendas_gdf.to_crs(CRS_OBJETIVO)
    print(f"CRS de tiendas_gdf: {tiendas_gdf.crs}")
except Exception as e:
    print(f"Error al cargar '{RUTA_TIENDAS_CON_CENSO_DENUE_INPUT}': {e}")
    raise

In [ ]:
# --- 2. Cargar y Preparar Datos de POIs de OSM ---
try:
    print(f"\nCargando POIs de OSM desde: {SHP_OSM_POIS}")
    osm_pois_gdf = gpd.read_file(SHP_OSM_POIS)
    print(f"  {len(osm_pois_gdf)} POIs de OSM cargados.")
    # Filtrar por bounding box de tus tiendas para reducir tamaño si el shapefile es nacional
    # minx, miny, maxx, maxy = tiendas_gdf.geometry.total_bounds
    # osm_pois_gdf = osm_pois_gdf.cx[minx:maxx, miny:maxy]
    # print(f"  {len(osm_pois_gdf)} POIs de OSM después de filtrar por extensión de tiendas.")
    
    if osm_pois_gdf.crs.to_string().upper() != CRS_OBJETIVO.upper():
        print(f"  Reproyectando POIs de OSM a {CRS_OBJETIVO}...")
        osm_pois_gdf = osm_pois_gdf.to_crs(CRS_OBJETIVO)
    print(f"  CRS de POIs OSM: {osm_pois_gdf.crs}")
except FileNotFoundError:
    print(f"Error: Archivo de POIs OSM no encontrado en '{SHP_OSM_POIS}'.")
    raise
except Exception as e:
    print(f"Error al cargar o procesar POIs de OSM: {e}")
    raise

In [ ]:
# --- 3. Definir Categorías de POIs OSM y Filtrar ---
# Basado en la Sección 4.2 de tu investigación (valores de `fclass`)
# ¡¡¡REVISA Y AJUSTA ESTA LISTA SEGÚN TUS NECESIDADES!!!
categorias_pois_osm = {
    'paradas_autobus': ['bus_stop', 'bus_station'],
    'estaciones_tren_metro': ['railway_station', 'subway_entrance', 'tram_stop', 'halt'],
    'atm_cajeros': ['atm'],
    'bancos_osm': ['bank'], # Puede solaparse con DENUE, pero OSM puede tener más ATMs
    'parques_recreacion': ['park', 'playground', 'sports_centre', 'pitch', 'leisure_centre'],
    'escuelas_osm': ['school', 'kindergarten'], # OSM puede tener guarderías ('kindergarten')
    'universidades_colegios_osm': ['university', 'college']
    # Añade más si son relevantes y OSM los mapea bien (ej. 'fuel' para gasolineras si DENUE no fue suficiente)
}

gdfs_pois_osm_filtrados = {}
print("\nFiltrando POIs de OSM por categorías...")
for categoria, fclass_values in categorias_pois_osm.items():
    temp_gdf = osm_pois_gdf[osm_pois_gdf[COL_OSM_FCLASS].isin(fclass_values)]
    gdfs_pois_osm_filtrados[categoria] = temp_gdf
    print(f"  Categoría OSM '{categoria}': {len(temp_gdf)} POIs encontrados.")

In [ ]:
# --- 4. Ingeniería de Características: Contar PDIs OSM en Radios ---
print("\nCalculando conteo de PDIs de OSM en radios...")
for radio_m in RADIOS_ANALISIS_M:
    print(f"  Procesando para radio de {radio_m} metros...")
    tiendas_buffer_gdf = tiendas_gdf.copy()
    tiendas_buffer_gdf['geometry_buffer'] = tiendas_gdf.geometry.buffer(radio_m)
    tiendas_buffer_gdf = tiendas_buffer_gdf.set_geometry('geometry_buffer')

    for categoria, pdi_gdf_cat in tqdm(gdfs_pois_osm_filtrados.items(), desc=f"Cat. OSM (radio {radio_m}m)"):
        nombre_col_conteo = f'osm_conteo_{categoria}_{radio_m}m'
        if not pdi_gdf_cat.empty:
            try:
                 joined_conteo = gpd.sjoin(tiendas_buffer_gdf[['TIENDA_ID', 'geometry_buffer']], 
                                      pdi_gdf_cat[['geometry']], 
                                      how='left', predicate='intersects')
            except TypeError:
                 joined_conteo = gpd.sjoin(tiendas_buffer_gdf[['TIENDA_ID', 'geometry_buffer']], 
                                      pdi_gdf_cat[['geometry']], 
                                      how='left', op='intersects')
            
            conteo_por_tienda = joined_conteo.dropna(subset=['index_right']).groupby('TIENDA_ID').size()
            tiendas_gdf[nombre_col_conteo] = tiendas_gdf['TIENDA_ID'].map(conteo_por_tienda).fillna(0).astype(int)
        else:
            tiendas_gdf[nombre_col_conteo] = 0
        # print(f"    Columna creada: {nombre_col_conteo}, suma: {tiendas_gdf[nombre_col_conteo].sum()}")

In [ ]:
# --- 5. Cargar y Preparar Datos de Red Vial de OSM ---
try:
    print(f"\nCargando red vial de OSM desde: {SHP_OSM_ROADS}")
    osm_roads_gdf = gpd.read_file(SHP_OSM_ROADS)
    print(f"  {len(osm_roads_gdf)} segmentos viales de OSM cargados.")
    
    # Filtrar por bounding box si es necesario y reproyectar
    # minx, miny, maxx, maxy = tiendas_gdf.geometry.total_bounds
    # osm_roads_gdf = osm_roads_gdf.cx[minx:maxx, miny:maxy] 
    # print(f"  {len(osm_roads_gdf)} segmentos viales después de filtrar por extensión.")

    if osm_roads_gdf.crs.to_string().upper() != CRS_OBJETIVO.upper():
        print(f"  Reproyectando red vial de OSM a {CRS_OBJETIVO}...")
        osm_roads_gdf = osm_roads_gdf.to_crs(CRS_OBJETIVO)
    print(f"  CRS de red vial OSM: {osm_roads_gdf.crs}")
except FileNotFoundError:
    print(f"Error: Archivo de red vial OSM no encontrado en '{SHP_OSM_ROADS}'. Omitiendo características viales.")
    osm_roads_gdf = None # Para que el código no falle después
except Exception as e:
    print(f"Error al cargar o procesar red vial de OSM: {e}")
    osm_roads_gdf = None
    

In [ ]:
# --- 6. Ingeniería de Características: Red Vial OSM ---
if osm_roads_gdf is not None and not osm_roads_gdf.empty:
    print("\nCalculando características de red vial OSM...")
    # Filtrar tipos de carreteras principales (ejemplo, ajusta según tu investigación)
    tipos_via_principal = ['motorway', 'trunk', 'primary', 'secondary']
    # Podrías querer añadir 'tertiary' si son relevantes en tus zonas urbanas
    
    vias_principales_gdf = osm_roads_gdf[osm_roads_gdf[COL_OSM_FCLASS].isin(tipos_via_principal)]
    print(f"  Número de segmentos de vías principales: {len(vias_principales_gdf)}")

    if not vias_principales_gdf.empty:
        # Distancia a la vía principal más cercana
        # Este cálculo es de punto a línea, puede ser lento.
        # La optimización con sindex es más compleja para línea-punto que para punto-punto.
        # Usaremos progress_apply para ver el progreso.
        print("  Calculando distancia a la vía principal más cercana (puede tardar)...")
        
        # Para calcular la distancia de un punto a un conjunto de líneas, 
        # podemos iterar o encontrar una forma más vectorizada.
        # nearest_points nos da los dos puntos más cercanos entre una geometría y otra colección.
        # Aquí queremos la distancia mínima de cada tienda a CUALQUIER vía principal.
        
        # Unir todas las vías principales en una sola MultiLineString o GeometryCollection para eficiencia
        # si se usa nearest_points con una sola geometría objetivo.
        # O iterar si el conjunto de vías principales no es demasiado grande.
        
        # Enfoque simple iterativo (puede ser lento para muchas tiendas / muchas vías)
        # dist_via_principal = []
        # all_vias_principales_unary = vias_principales_gdf.unary_union # Unir todas las líneas en una sola geometría
        # for tienda_geom in tqdm(tiendas_gdf.geometry, desc="  Dist. a vía principal"):
        #     dist = tienda_geom.distance(all_vias_principales_unary)
        #     dist_via_principal.append(dist)
        # tiendas_gdf['osm_dist_via_principal_m'] = dist_via_principal

        # Enfoque más eficiente si rtree está instalado:
        try:
            sindex_vias_principales = vias_principales_gdf.sindex
            dist_via_principal = []
            for idx_tienda, tienda_row in tqdm(tiendas_gdf.iterrows(), total=len(tiendas_gdf), desc="  Dist. a vía principal"):
                tienda_geom = tienda_row.geometry
                # Buscar posibles vías cercanas en un buffer grande para reducir candidatos
                buffer_dist_busqueda = max(RADIOS_ANALISIS_M) * 5 # ej. 5km
                posibles_indices = list(sindex_vias_principales.intersection(tienda_geom.buffer(buffer_dist_busqueda).bounds))
                if not posibles_indices:
                    dist_via_principal.append(99999.0) # Valor grande si no hay vías en el buffer de búsqueda
                    continue
                
                candidatas_vias_gdf = vias_principales_gdf.iloc[posibles_indices]
                if candidatas_vias_gdf.empty:
                    dist_via_principal.append(99999.0)
                    continue

                dist = candidatas_vias_gdf.geometry.distance(tienda_geom).min()
                dist_via_principal.append(dist)
            
            tiendas_gdf['osm_dist_via_principal_m'] = dist_via_principal
            tiendas_gdf['osm_dist_via_principal_m'] = tiendas_gdf['osm_dist_via_principal_m'].fillna(99999).astype(float)
            print(f"    Columna 'osm_dist_via_principal_m' creada. Promedio: {tiendas_gdf['osm_dist_via_principal_m'][tiendas_gdf['osm_dist_via_principal_m'] < 99999].mean():.2f} m")
        except Exception as e_dist_via:
            print(f"    Error calculando distancia a vía principal con sindex: {e_dist_via}. Intenta instalar 'rtree'.")
            tiendas_gdf['osm_dist_via_principal_m'] = 99999.0


        # Contar tipos de vías en un radio (más complejo, podría requerir sjoin con buffer y luego value_counts)
        # Ejemplo: tipo de vía más común en un radio de 200m
        # ... (esto puede ser demasiado para un datathon rápido) ...
    else:
        print("  No se encontraron vías principales con los fclass especificados.")
        tiendas_gdf['osm_dist_via_principal_m'] = 99999.0
else:
    print("\nNo se cargó la red vial de OSM, se omiten características viales.")
    tiendas_gdf['osm_dist_via_principal_m'] = 99999.0 # Añadir columna con valor por defecto

In [ ]:
# --- 7. Guardar el Resultado ---
print("\n--- Guardando GeoDataFrame Enriquecido con OSM ---")
print("Primeras filas del dataset final con datos OSM:")
print(tiendas_gdf.head())
print("\nColumnas finales:")
print(tiendas_gdf.columns.tolist())

try:
    tiendas_gdf.to_file(ARCHIVO_SALIDA_GPKG, driver='GPKG', layer='tiendas_censo_denue_osm')
    print(f"\nDataset enriquecido guardado como GeoPackage en '{ARCHIVO_SALIDA_GPKG}'")
except Exception as e:
    print(f"Error al guardar el archivo de salida GeoPackage: {e}")

print("\n--- Integración de Datos de OpenStreetMap Completada ---")